In [ ]:
# https://yobro.tistory.com/142

In [ ]:
import requests
import pandas as pd
import python_quant_super_value as pq
import time
import bs4
import numpy as np
import urllib.parse
import pandas as pd
import matplotlib.pyplot as plt
import numpy
from pykrx import stock
from datetime import datetime

In [ ]:
import imp
import python_quant_super_value
imp.reload(python_quant_super_value)

## 전 종목 코드 구해 오기 (Python)

In [ ]:
# 전종목 시가총액 구하기 (https://yobro.tistory.com/142)

In [ ]:
today = datetime.today().strftime("%Y%m%d")
df_kospi = stock.get_market_cap_by_ticker(today, market='KOSPI')
df_kosdaq = stock.get_market_cap_by_ticker(today, market='KOSDAQ')
code_data = pd.concat([df_kospi, df_kosdaq], axis = 0)
code_data['기업명'] = [stock.get_market_ticker_name(ticker) for ticker in code_data.index]
code_data = code_data[code_data.거래량 > 0] # 거재정지 종목 제외
code_data = code_data[['기업명', '종가', '시가총액']]
code_data.index = 'A' + code_data.index
code_data.index.name = '종목코드'

## 시총 하위 20%

In [ ]:
def get_cap_range(value_df, ratio=[0.8, 1.0]):   
    temp_df = value_df
    temp_df['시가총액'] = pd.to_numeric(temp_df['시가총액'])
    temp_df = temp_df.sort_values(by='시가총액', ascending=False)
    temp_df['시가총액비율'] = temp_df['시가총액'].rank(ascending=False) / temp_df.shape[0]
    sorted_cap_value = temp_df[(temp_df['시가총액비율'] > min(ratio)) & (temp_df['시가총액비율'] <= max(ratio))]
    return sorted_cap_value  

In [ ]:
code_data_filter = get_cap_range(code_data, ratio=[0.80, 1.0])
code_data_filter = code_data_filter[['기업명']]

In [ ]:
code_data_filter.shape[0]

## fnguide 데이터 수집하기

In [ ]:
# [코드 3.27/29/31] 모든 종목에 대해서 재무제표, 재무비율, 투자지표 데이터 가져오기 (CH3. 데이터 수집하기.ipynb)
total_value = pd.DataFrame()
for num, code in enumerate(code_data_filter.index):
    if num % 10 == 0:
        print(code, num+1, "/", code_data_filter.shape[0])
    try:
        time.sleep(1)        
        try:
            cap = pq.make_cap_dataframe(code)
            fs_df = pq.make_fs_dataframe(code)
            fhd_df = pq.make_fhd_dataframe(code)
        except requests.exceptions.Timeout:
            time.sleep(60)
            cap = pq.make_cap_dataframe(code)
            fs_df = pq.make_fs_dataframe(code)
            fhd_df = pq.make_fhd_dataframe(code)
        except ValueError:
            continue
        except KeyError:
            continue
        except UnboundLocalError:
            continue

        value = pd.merge(cap, fs_df, how='outer', right_index=True, left_index=True)
        value = pd.merge(value, fhd_df, how='outer', right_index=True, left_index=True)
        value['1/PSR'] = float(value['매출액'][0]) / float(value['시가총액'][0])
        value['1/PER'] = float(value['순이익'][0]) / float(value['시가총액'][0])
        value['1/PBR'] = float(value['자본'][0]) / float(value['시가총액'][0])
        value['1/PCR'] = float(value['영업현금흐름'][0]) / float(value['시가총액'][0])
        value = pd.merge(code_data_filter.loc[[code]], value, how='outer', right_index=True, left_index=True)
        if num == 0 :
            total_value = value
        else:
            total_value = pd.concat([total_value, value])
            
        if num + 1 == code_data_filter.shape[0]:
            print("Analysis Done")

    except ValueError:
        continue
    except KeyError:
        continue
    except UnboundLocalError:
        continue


In [ ]:
total_value['적정가'] = round((total_value['BPS'].astype('float')*(1+total_value['ROE3년']*0.01-0.05*((100+total_value['부채비율3년'])/100)**0.5)**10).astype(float), -1)

In [ ]:
total_value.to_excel(r'data\total_value_{}.xlsx'.format(today))
total_value.shape[0]

## super value 전략 구현하기

In [ ]:
fs_path = r'data\total_value_{}.xlsx'.format(today)
value_df = pd.read_excel(fs_path)
value_df = value_df.set_index('종목코드')
value_list = ['1/PSR','1/PER','1/PBR','1/PCR']
value_combo = pq.value_combo(value_df, value_list, 50)

In [ ]:
value_combo.to_excel(r'data\value_combo_{}.xlsx'.format(today))

In [ ]:
value_combo